In [60]:
REALTIME_ESTIMATOR = False
WEEK = 67

In [61]:
import os
import json
import time
from datetime import datetime
import requests
import numpy as np
import pandas as pd
from web3 import Web3

from urllib.request import urlopen

In [62]:
reports_dir = f'reports/{WEEK}'

if not os.path.exists(reports_dir):
    os.mkdir(reports_dir)

def get_export_filename(network, token):
    return f'{reports_dir}/__{network}_{token}_subgraph.json'

In [63]:
week_1_start_ts = 1590969600
week_end_timestamp = week_1_start_ts + WEEK * 7 * 24 * 60 * 60
week_start_timestamp = week_end_timestamp - 7 * 24 * 60 * 60

BAL_addresses = {
    1: '0xba100000625a3754423978a60c9317c58a424e3d',
    137: '0x9a71012b13ca4d3d0cdc72a177df3ef03b0e76a3',
    42161: '0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8'
}

networks = {
#     1: 'ethereum',
#     137: 'polygon',
    42161: 'arbitrum'
}

CLAIM_PRECISION = 12 # leave out of results addresses that mined less than CLAIM_THRESHOLD
CLAIM_THRESHOLD = 10**(-CLAIM_PRECISION)

In [64]:
if REALTIME_ESTIMATOR:
    url = 'https://raw.githubusercontent.com/balancer-labs/bal-mining-scripts/master/config/redirect.json'
    jsonurl = urlopen(url)
    redirects = json.loads(jsonurl.read())
else:
    redirects = json.load(open('config/redirect.json'))

In [65]:
def get_start_block(network, start_timestamp):
    
    endpoint = {
        1: 'https://api.thegraph.com/subgraphs/name/blocklytics/ethereum-blocks',
        137: 'https://thegraph.com/legacy-explorer/subgraph/matthewlilley/polygon-blocks',
        42161: 'https://api.thegraph.com/subgraphs/name/ianlapham/arbitrum-one-blocks'
    }
    
    query = '''
        {
          blocks(first: 1, orderBy: number, orderDirection: desc, where: { timestamp_lte: {}}) {
            number
          }
        }
    '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
        start_timestamp,
    )
    
    r = requests.post(endpoint[network], json = {'query': query})

    try:
        start_block = json.loads(r.content)['data']['blocks'][0]['number']
    except:
        raise Exception(json.loads(r.content)['errors'][0]['message'])
    
    return start_block

In [66]:
def get_pool_share_subgraph(pools_addresses, network, time_travel_block):
    
    endpoint = {
        1: 'https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2',
        137: '',
        42161: 'https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-arbitrum-v2'
    }

    query = '''
        {
          poolShares(
            first: 1000,
            block: { number: {} },
            where: { poolId_in: ["{}"]}) {
              userAddress {
                id
              }
              poolId {
                id
              }
              balance
            }
        }
    '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
        time_travel_block,
        '","'.join(pools_addresses)
    )
    r = requests.post(endpoint[network], json = {'query':query})
    try:
        p = json.loads(r.content)['data']['poolShares']
    except:
        raise Exception(json.loads(r.content)['errors'][0]['message'])
    
    BPT_supply_df = pd.DataFrame(p)
    BPT_supply_df['block'] = int(time_travel_block)
    BPT_supply_df['userAddress'] = BPT_supply_df['userAddress'].map(lambda x: x['id'])
    BPT_supply_df['poolId'] = BPT_supply_df['poolId'].map(lambda x: x['id'])
    BPT_supply_df['balance'] = BPT_supply_df['balance'].astype(float)
    BPT_supply_df['timestamp'] = int(week_start_timestamp)
    
    return BPT_supply_df

In [67]:
def get_bpt_tranfers_subgraph(pools_addresses, network, start_timestamp, end_timestamp):

    endpoint = {
        1: '',
        137: '',
        42161: 'https://api.thegraph.com/subgraphs/name/mendesfabio/arb-bal'
    }
    
    query = '''
        {
          transfers(
              first: 1000, 
              orderBy: block, 
              where: { timestamp_gte: {}, timestamp_lt: {}, pool_in: ["{}"] }) {
            pool {
              id
            }
            fromAddress
            toAddress
            amount
            block
            timestamp
          } 
        }
    '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
        start_timestamp,
        end_timestamp,
        '","'.join(pools_addresses)
    )
    r = requests.post(endpoint[network], json = {'query':query})
    try:
        p = json.loads(r.content)['data']['transfers']
    except:
        raise Exception(json.loads(r.content)['errors'][0]['message'])
    
    df1 = pd.DataFrame(p)

    query = '''
        {
          transfers(
              first: 1000, 
              skip: 1000,
              orderBy: block, 
              where: { timestamp_gte: {}, timestamp_lt: {}, pool_in: ["{}"] }) {
            pool {
              id
            }
            fromAddress
            toAddress
            amount
            block
            timestamp
          } 
        }
    '''.replace('{','{{').replace('}','}}').replace('{{}}','{}').format(
        start_timestamp,
        end_timestamp,
        '","'.join(pools_addresses)
    )
    r = requests.post(endpoint[network], json = {'query':query})
    try:
        p = json.loads(r.content)['data']['transfers']
    except:
        raise Exception(json.loads(r.content)['errors'][0]['message'])
    
    df2 = pd.DataFrame(p)
    
    BPT_supply_df = pd.concat([df1, df2])
    BPT_supply_df['poolId'] = BPT_supply_df['pool'].map(lambda x: x['id'])
    BPT_supply_df['amount'] = BPT_supply_df['amount'].astype(float)
    BPT_supply_df['timestamp'] = BPT_supply_df['timestamp'].astype(int)
    BPT_supply_df['block'] = BPT_supply_df['block'].astype(int)
    
    df1 = BPT_supply_df.copy()
    df1['userAddress'] = df1['fromAddress']
    df1['balance'] = - df1['amount']
    df1 = df1[['userAddress', 'poolId', 'block', 'timestamp', 'balance']]
    
    df2 = BPT_supply_df.copy()
    df2['userAddress'] = df2['toAddress']
    df2['balance'] = df2['amount']
    df2 = df2[['userAddress', 'poolId', 'block', 'timestamp', 'balance']]
    
    return df1.append(df2)

In [68]:
V2_LM_ALLOCATION_URL = 'https://raw.githubusercontent.com/balancer-labs/frontend-v2/master/src/lib/utils/liquidityMining/MultiTokenLiquidityMining.json'
jsonurl = urlopen(V2_LM_ALLOCATION_URL)

try:
    V2_ALLOCATION_THIS_WEEK = json.loads(jsonurl.read())[f'week_{WEEK}']
except KeyError:
    V2_ALLOCATION_THIS_WEEK = {}

full_export = pd.DataFrame()

for chain in V2_ALLOCATION_THIS_WEEK:
    print('------------------------------------------------------------------------------')
    print('\nChain: {}'.format(chain['chainId']))
    if chain['chainId'] == 42161:
        df = pd.DataFrame()
        for pool,rewards in chain['pools'].items():
            for r in rewards:
                pool_address = pool.lower()
                df.loc[pool_address,r['tokenAddress']] = r['amount']
        if len(df) == 0:
            print('No incentives for this chain')
            continue
        df.fillna(0, inplace=True)
        df.index.name = 'pool_address'
        bal_address = BAL_addresses[chain['chainId']]
        if bal_address in df.columns:
            bal_on_this_chain = df[bal_address].sum()
        else:
            bal_on_this_chain = 0
        print('BAL to be mined on this chain: {}'.format(bal_on_this_chain))
        
        start_block = get_start_block(chain['chainId'], week_start_timestamp)
        shares = get_pool_share_subgraph(df.index, chain['chainId'], start_block)
        transfers = get_bpt_tranfers_subgraph(df.index, chain['chainId'], week_start_timestamp, week_end_timestamp)
    else:
        pass

------------------------------------------------------------------------------

Chain: 1
------------------------------------------------------------------------------

Chain: 137
------------------------------------------------------------------------------

Chain: 42161
BAL to be mined on this chain: 6000.0


In [69]:
# merge and cumsum

df_bpts = shares.append(transfers)

df_bpts = df_bpts[df_bpts['userAddress'] != '0x0000000000000000000000000000000000000000']
df_bpts = df_bpts[df_bpts['userAddress'] != '0xBA12222222228d8Ba445958a75a0704d566BF2C8']

df_bpts = df_bpts.sort_values(by='block')

df_bpts = df_bpts.groupby(['block', 'timestamp', 'userAddress', 'poolId']).sum().groupby(level=[2, 3]).cumsum().reset_index()

# fill balance gaps

blocks_pools = df_bpts[['block', 'timestamp', 'poolId']].drop_duplicates()
lps_pools = df_bpts[['userAddress', 'poolId']].drop_duplicates()

util = pd.merge(lps_pools, blocks_pools, on='poolId', how='inner')

running_bpts = util.merge(df_bpts, on=['poolId', 'userAddress', 'block', 'timestamp'], how='left')

running_bpts["balance"] = running_bpts.groupby(['userAddress', 'poolId'])['balance'].transform(lambda x: x.ffill().fillna(0))

# calc timestamp diffs

running_bpts = running_bpts.sort_values(['block'])
running_bpts['duration'] = abs(running_bpts.groupby(['userAddress', 'poolId'])['timestamp'].diff(-1))

running_bpts.loc[pd.isnull(running_bpts['duration']), 'duration'] = week_end_timestamp - running_bpts[pd.isnull(running_bpts['duration'])]['timestamp']


In [70]:
# foo = util.merge(df_bpts, on=['poolId', 'userAddress', 'block', 'timestamp'], how='left')
# foo = foo.sort_values(by=['userAddress', 'poolId', 'block'])
# foo[foo['block'] == foo['block'].min()] = foo[foo['block'] == foo['block'].min()].fillna(0)
# foo = foo.ffill()

In [71]:
# running_bpts = util.merge(df_bpts, on=['poolId', 'userAddress', 'block', 'timestamp'], how='left')
# running_bpts["balance"] = running_bpts.groupby(['userAddress', 'poolId'])['balance'].transform(lambda x: x.ffill().fillna(0))

# foo.equals(running_bpts.sort_values(by=['userAddress', 'poolId', 'block']))

In [72]:
running_bpts['share'] = running_bpts['balance'] / running_bpts.groupby(['block', 'poolId'])['balance'].transform('sum')
running_bpts['share_integral'] = running_bpts['share'] * running_bpts['duration']

integrator = running_bpts.groupby(['poolId'])['share_integral'].sum().reset_index()
integrator = integrator.rename(columns={'share_integral': 'integral'})

In [73]:
tw_share = running_bpts.groupby(['userAddress', 'poolId'])['share_integral'].sum().reset_index()

tw_share = tw_share.merge(integrator, on='poolId')

tw_share['tw_share'] = tw_share['share_integral'] / tw_share['integral']

tw_share = tw_share[['userAddress', 'poolId', 'tw_share']]
tw_share = tw_share.rename(columns={'userAddress': 'miner', 'poolId': 'pool_address'})

## V2 LM

In [74]:
network_name = 'arbitrum'

pools_addresses_and_tokens_earned = df.copy()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' - Querying Subgraph for the V2 LPs...')

BPT_share_df = tw_share.copy()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' - Done!')
BPT_share_df['miner'] = BPT_share_df['miner'].apply(Web3.toChecksumAddress)
BPT_share_df.set_index(['pool_address','miner'], inplace=True)

bal_mined_v2 = pools_addresses_and_tokens_earned.mul(BPT_share_df['tw_share'], axis=0)

if REALTIME_ESTIMATOR:
    bal_mined_v2 *= week_passed

miner_export = bal_mined_v2.groupby('miner').sum()

for token in miner_export.columns:
    miner_export_v2 = miner_export[token].dropna()
    print(f'\n{miner_export_v2.sum()} {token} mined on {network_name}')

    v2_miners = pd.DataFrame(miner_export_v2).reset_index()
    n = len(v2_miners['miner'][v2_miners['miner'].isin(redirects.keys())])
    print(f'Redirect: {n} redirectors found')
    v2_miners['miner'] = v2_miners['miner'].apply(lambda x: redirects.get(x,x))
    miner_export_v2 = v2_miners.groupby('miner').sum()[token]

    if not REALTIME_ESTIMATOR:
        filename = get_export_filename(network_name, token)
        print(filename)
        (
            miner_export_v2[miner_export_v2>=CLAIM_THRESHOLD]
            .apply(
                lambda x: format(
                    x, 
                    f'.{CLAIM_PRECISION}f'
                )
            )
            .to_json(filename, indent=4)
        )

miner_export

2021-09-22 17:05:32 - Querying Subgraph for the V2 LPs...
2021-09-22 17:05:32 - Done!

6000.000000000035 0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8 mined on arbitrum
Redirect: 0 redirectors found
reports/67/__arbitrum_0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8_subgraph.json


,0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8
miner,
0x0002208085F5e1946500aD6616aA0Ee2f19e826d,0.023104
0x0034daf2e65F6ef82Bc6F893dbBfd7c232a0e59C,182.337118
0x00645Dd21310882cC32399abCb54e0a05b3b5D1d,0.090921
0x016c44A85cd6cFBBe06b15984b84b77F82c62f22,0.001271
0x01CC7BA4F67e102e8f789C2859E091CFcA7Bde35,0.013803
...,...
0xf928400344eC91B7e728B428C68E8F0136Bb984c,0.008046
0xfC86F1156b283b16eDB7505356F98Ea0d2Cda341,0.005819
0xfCc1b0AAEb2b9063AB34FdfBB3EEDc2FF30ae4F2,3.325301


## Results

In [76]:
with open('reports/67/__arbitrum_0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8_subgraph.json', 'r') as f:
    data = json.load(f)
    
subgraph = pd.DataFrame({'subgraph': data})

with open('reports/67/__arbitrum_0x040d1edc9569d4bab2d15287dc5a4f10f56a56b8.json', 'r') as f:
    data = json.load(f)
    
bigquery = pd.DataFrame({'bigquery': data})

results = subgraph.join(bigquery)

results['subgraph'] = results['subgraph'].astype(float)
results['bigquery'] = results['bigquery'].astype(float)

results['diff'] = abs(results['bigquery'] - results['subgraph'])

In [77]:
results.sort_values(by='diff')

,subgraph,bigquery,diff
0x87105752954887b1140b7cea133eacC8AE8607b5,4.591320e-07,4.591810e-07,4.900000e-11
0xE8Aa67AfC6eC84150140a2b8A62AD98F60EC7af1,1.770681e-06,1.770868e-06,1.870000e-10
0x9f13365F0d9C0470b86817E05a9c0Cca4D17C8Ae,2.539464e-06,2.539733e-06,2.690000e-10
0xab1F5Ec482414170f1520152a9AcC60290DFbA87,2.749907e-06,2.750349e-06,4.420000e-10
0x7c15f73f813b678b34b762119913E1c009512350,7.572251e-06,7.573052e-06,8.010000e-10
...,...,...,...
0xCAab2680d81dF6b3e2EcE585bB45cEe97BF30cD7,9.429931e+01,9.428386e+01,1.544353e-02
0x7c132BdE28AD1e0101D45121757d7c6C2e8A550a,1.138616e+02,1.138325e+02,2.905303e-02
0xf3f5C252e8ACd60671f92c7F72cf33661221Ef42,1.155844e+02,1.155379e+02,4.646839e-02
0x6A025e093C6C0c5381d5763283E1dB339ff22Eb6,2.659719e+03,2.659892e+03,1.724860e-01


In [78]:
results[results['diff'] > 0 ]

,subgraph,bigquery,diff
0x0002208085F5e1946500aD6616aA0Ee2f19e826d,0.023104,0.023093,1.078220e-05
0x0034daf2e65F6ef82Bc6F893dbBfd7c232a0e59C,182.337118,182.342527,5.409163e-03
0x00645Dd21310882cC32399abCb54e0a05b3b5D1d,0.090921,0.090931,9.622340e-06
0x016c44A85cd6cFBBe06b15984b84b77F82c62f22,0.001271,0.001269,1.196566e-06
0x01CC7BA4F67e102e8f789C2859E091CFcA7Bde35,0.013803,0.013798,4.931480e-06
...,...,...,...
0xf928400344eC91B7e728B428C68E8F0136Bb984c,0.008046,0.008037,9.361495e-06
0xfC86F1156b283b16eDB7505356F98Ea0d2Cda341,0.005819,0.005815,3.840447e-06
0xfCc1b0AAEb2b9063AB34FdfBB3EEDc2FF30ae4F2,3.325301,3.325266,3.487203e-05
0xfD102a750C595d95ACE7bDBeD17E8b444891610d,0.016950,0.016951,8.421380e-07
